In [2]:
import codecs
import nltk.data
import pymorphy2
import re
import os
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
from itertools import takewhile
morph = pymorphy2.MorphAnalyzer()

In [327]:
delims = ",.!?/&:;@'*()«»...\n"
stopwords = open("stopwords.txt", 'r').read().split("\n")

# Delete delimiters, remove stopwords, make float . stay as _
def process(text):
    title = text.split("\n")[0]
    text = re.sub(r'\[.+?\]\s*', ' ', text)
    text = re.sub(r'«.+?»\s*', ' ', text)
    text = re.sub(r'-', ' ', text)
    text = re.sub(r' — ', ' . ', text)
    text = re.sub(r'([+-]?[0-9]+)\.([0-9]+)',r'\1_\2', text)
    text = re.sub(r'([+-]?[0-9]+),([0-9]+)',r'\1_\2', text)
    lines = [w.strip() for w in re.split("["+"\\".join(delims)+"]", text.lower()) if w != ""]
    return title, [' '.join(filter(lambda x: x.lower() not in stopwords, text.split())) for text in lines]
 
# if there is n-gram keyword with high certanity, delete related n-1-grams
def remove_dups(less, more, num):
    words = [x[0] for x in less]
    todel = []
    for s in more:
        l = s[0].split(' ')
        cnt = 0
        for i in range(len(l) - num + 1):
            if not " ".join(l[i: i + num]).strip() in words:
                break
            cnt = cnt + 1
        if cnt == 2:
            todel += [" ".join(l[i: i + num]) for i in range(len(l) - num + 1)]
    return list(filter(lambda s: s[0] not in todel, less))

# check if difference is only in wordforms
def are_alike(s_a, s_b):
    a = s_a.split(' ')
    b = s_b.split(' ')
    for i in range(len(a)):
        if morph.parse(a[i])[0].normal_form != morph.parse(b[i])[0].normal_form:
            return False
    return True

# remove keywords that differ in wordform only
def remove_wordforms(l):
    new_l = []
    for a in range(len(l)):
        flag = True
        for b in range(a):
            if a != b:
                if are_alike(l[a], l[b]):
                    flag = False
                    break
        if flag:
            new_l.append(l[a])
    return new_l

In [328]:
text = [process(open("wiki/" + f, 'r').read())
        for f in os.listdir("wiki")
        if f.endswith(".txt")]

In [329]:
with open('out.txt','w') as f:
    for (title, data) in text:    
        f.write(title + "\n")
        print(title)
        total = 0.0
        monograms = []
        bigrams = []
        trigrams = []
        fourgrams = []
        fivegrams = []
        # get ngrams
        for line in data:
            line_len = len(line.split(" "))
            total += line_len

            token = nltk.word_tokenize(line)
            monograms += ngrams(token, 1)
            if line_len < 2:
                continue
            bigrams += ngrams(token, 2)
            if line_len < 3:
                continue
            trigrams += ngrams(token, 3)
            if line_len < 4:
                continue
            fourgrams += ngrams(token, 4)
            if line_len < 5:
                continue
            fivegrams += ngrams(token, 5)

        # get frequencies
        d1 = Counter(monograms)
        d2 = Counter(bigrams)
        d3 = Counter(trigrams)
        d4 = Counter(fourgrams)
        d5 = Counter(fivegrams)

        # get probabilities of occurence of ngram in text, choose keywords based on probability and frequence
        key1 = sorted([(" ".join(x), float(d1[x] / total)) 
                for x in d1 if float(d1[x] / total) > 0.01], key=lambda tup: tup[1], reverse=True)[:5]

        key2 = sorted([(" ".join(x), d2[x] * d2[x] / float(d1[(x[0],)])) 
                for x in d2 if d2[x] > 1], key=lambda tup: tup[1], reverse=True)[:15]
        key1 = remove_dups(key1, key2, 1)

        key3 = sorted([(" ".join(x), d3[x] * d3[x] / float(d2[(x[0], x[1],)])) 
                for x in d3 if d3[x] > 1], key=lambda tup: tup[1], reverse=True)[:10]
        key2 = remove_dups(key2, key3, 2)

        key4 = sorted([(" ".join(x), d4[x] * d4[x] / float(d3[(x[0], x[1], x[2],)])) 
                for x in d4 if d4[x] > 1], key=lambda tup: tup[1], reverse=True)[:5]
        key3 = remove_dups(key3, key4, 3)    

        key5 = sorted([(" ".join(x), d5[x] * d5[x] / float(d4[(x[0], x[1], x[2], x[3],)])) 
                for x in d5 if d5[x] > 1], key=lambda tup: tup[1], reverse=True)[:5]
        key4 = remove_dups(key4, key5, 4)
        key1 = remove_wordforms([x[0] for x in key1])
        key2 = remove_wordforms([x[0] for x in key2])
        key3 = remove_wordforms([x[0] for x in key3])
        key4 = remove_wordforms([x[0] for x in key4])
        key5 = remove_wordforms([x[0] for x in key5])
        keywords = key1 + key2 + key3 + key4 + key5
        print(keywords)
        for string in keywords:
            f.write(string + ", ")
        f.write("\n\n")
        
        print("\n\n")

Android 
['android', 'google', 'операционной системы', 'исходный код', 'ос android', 'material design', 'начиная версии', 'небольшого обновления', 'марте 2012', 'open handset alliance', 'native development kit', 'операционную систему android', 'версия операционной системы', 'передачи идентификационной информации', '23 сентября 2008', '2013 google представила', 'android native development kit', 'android 4_0 ice cream sandwich', 'передачи идентификационной информации серверы компании']



API
['api', 'функции', 'операционных систем', 'семантика функции', 'типов аргументов', 'области видимости', 'низкоуровневых компонентов', 'объявления функции', 'синтаксической конструкции', 'включает описание', 'низкого уровня', 'графических интерфейсов', 'web api', 'системные вызовы ос', 'api операционных систем', 'api графических интерфейсов', 'используют api низкоуровневых компонентов']



Java Virtual Machine
['java', 'jvm', 'sun', 'microsoft', 'байт', 'virtual machine', 'байт кода', 'спецификации j